In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from ripser import ripser
from dowker_homology.persistence import Persistence
import gudhi
from persim import plot_diagrams

from persistence_benchmarks.benchmark_data import PersistenceDiagramBenchmark
from persistence_benchmarks.noise import normal_noise

%matplotlib inline

In [ ]:
# benchmark
pb = PersistenceDiagramBenchmark(shape='circle', 
                                 sampling='noise', 
                                 noise_distribution=normal_noise(sd=0.1))

In [ ]:
# generate benchmark data
X = pb.sample(n=100)
target = pb.target()

In [ ]:
# calculate diagrams from data
extrinsic_dimension = X.shape[1]
dimension = 1
# rips complex
dgms_rips = ripser(X, maxdim=dimension)['dgms']
# ambient Cech complex
dgms_dh = Persistence(dissimilarity='ambient', dimension=dimension).persistent_homology(X)
# alpha complex
persistence_alpha = gudhi.AlphaComplex(points=X).create_simplex_tree().persistence()
dgms_alpha = [np.array([p[1] for p in persistence_alpha if p[0]==0]), 
              np.array([p[1] for p in persistence_alpha if p[0]==1])]

In [ ]:
# bottleneck distances
bottleneck_distances = {'rips': [], 
                        'dh': [], 
                        'alpha': []}
for i in range(2):
    bottleneck_distances['rips'].append(gudhi.bottleneck_distance(target[i], dgms_rips[i]))
    bottleneck_distances['dh'].append(gudhi.bottleneck_distance(target[i], dgms_dh[i]))
    bottleneck_distances['alpha'].append(gudhi.bottleneck_distance(target[i], dgms_alpha[i]))
    
print(pd.DataFrame(bottleneck_distances))

In [ ]:
# plot persistence diagrams
plt.subplots(figsize = (10, 10))
for ix, dgm in enumerate([target, dgms_rips, dgms_dh, dgms_alpha]):
    plt.subplot(2, 2, ix + 1)
    plot_diagrams(dgm)